# this notebook shows backtestlib.optimizer.BenchMark etc.

In [1]:
import sys
sys.path.insert(0, "..")

import logging as log
log.basicConfig(level = log.INFO, stream=sys.stdout,\
                format='backtest-%(asctime)s-%(funcName)s: %(message)s', \
                datefmt='%Y-%b-%d %H:%M:%S')
from  backtestlib.optimizer import BenchMark, SampleBased, SpectralCut, SpectralSelection,ShrinkToIdentity,POET
import pandas as pd
import numpy as np
import numpy.linalg as LA
%load_ext autoreload
%autoreload 2

In [2]:
signal = pd.Series(data = [1,2,2,1.5,-.2],index = ['AAPL','GOOGLE','MMM','BAC','GS'])
signal

AAPL      1.0
GOOGLE    2.0
MMM       2.0
BAC       1.5
GS       -0.2
dtype: float64

In [3]:
c = 0.02
r = 0.00
n = 50
p = len(signal)

rand = np.random.RandomState(1233)
true_var = np.diag(np.ones(p))*(1-c) + c*np.ones((p,p))
true_mu = np.ones(p)*r

returns = rand.multivariate_normal(true_mu, true_var, n).T
sample_var =(returns@returns.T)/n
sample_var

array([[ 9.39108912e-01, -7.45041968e-03, -2.83134286e-02,
        -8.44307014e-03,  6.24612559e-04],
       [-7.45041968e-03,  8.62720972e-01, -2.20134863e-01,
         4.85521710e-02,  2.43102080e-01],
       [-2.83134286e-02, -2.20134863e-01,  7.16141932e-01,
         1.33318276e-02, -1.19808125e-01],
       [-8.44307014e-03,  4.85521710e-02,  1.33318276e-02,
         8.53522473e-01,  5.64317563e-02],
       [ 6.24612559e-04,  2.43102080e-01, -1.19808125e-01,
         5.64317563e-02,  9.83826046e-01]])

In [4]:
opt = BenchMark()
opt.optimize(signal = signal)

backtest-2021-Feb-28 22:52:48-__init__: BenchMark initialized


array([0.2, 0.2, 0.2, 0.2, 0.2])

 `optimize` method output the weight correspond to signal

In [5]:
opt = SampleBased()
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-28 22:52:48-__init__: SampleBased initialized


array([ 0.13203564,  0.36667792,  0.40506277,  0.16931761, -0.07309393])

 `get_weight_map` method output a `weight_map` such that `weight_map@ signal/ np.sum(weight_map@ signal)` equals `weight`

In [6]:
np.round(opt.get_weight_map(),3)

array([[ 1.067e+00,  2.100e-02,  4.800e-02,  9.000e-03, -1.000e-03],
       [ 2.100e-02,  1.336e+00,  3.650e-01, -6.300e-02, -2.820e-01],
       [ 4.800e-02,  3.650e-01,  1.528e+00, -5.100e-02,  9.900e-02],
       [ 9.000e-03, -6.300e-02, -5.100e-02,  1.180e+00, -5.800e-02],
       [-1.000e-03, -2.820e-01,  9.900e-02, -5.800e-02,  1.101e+00]])

In [7]:
opt = SpectralCut(delta = 0.6)
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-28 22:52:48-__init__: SpectralCut initialized with  delta 0.6
backtest-2021-Feb-28 22:52:48-get_weight_map: Optimal K is 5
backtest-2021-Feb-28 22:52:48-get_weight_map:  return component [-0.56292848  0.56166544 -1.79661716 -0.34753213  2.70352208]


array([ 0.13203564,  0.36667792,  0.40506277,  0.16931761, -0.07309393])

In [8]:
np.round(opt.get_weight_map(signal = signal,sample_var = sample_var),3)

backtest-2021-Feb-28 22:52:48-get_weight_map: Optimal K is 5
backtest-2021-Feb-28 22:52:48-get_weight_map:  return component [-0.56292848  0.56166544 -1.79661716 -0.34753213  2.70352208]


array([[ 1.067e+00,  2.100e-02,  4.800e-02,  9.000e-03, -1.000e-03],
       [ 2.100e-02,  1.336e+00,  3.650e-01, -6.300e-02, -2.820e-01],
       [ 4.800e-02,  3.650e-01,  1.528e+00, -5.100e-02,  9.900e-02],
       [ 9.000e-03, -6.300e-02, -5.100e-02,  1.180e+00, -5.800e-02],
       [-1.000e-03, -2.820e-01,  9.900e-02, -5.800e-02,  1.101e+00]])

In [9]:
opt = SpectralSelection(delta = 0.6, c = 10)
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-28 22:52:48-__init__: SpectralCut initialized with  delta 0.6, c 10
backtest-2021-Feb-28 22:52:48-optimize: Optimal gamma is 0.004662
backtest-2021-Feb-28 22:52:48-optimize:  return component [-0.56253782  0.55329965 -1.773227   -0.24565894  0.69021843]


array([ 0.20268037,  0.19992521,  0.21709359,  0.39463785, -0.01433701])

In [10]:
opt = ShrinkToIdentity()
opt.optimize(signal = signal,returns = returns)


backtest-2021-Feb-28 22:52:48-__init__: ShrinkToIdentity initialized
backtest-2021-Feb-28 22:52:48-optimize: shrink coefficients: a 1.708, b 0.3, d 1.734, m 0.858


array([ 0.1141222 ,  0.40128602,  0.42972691,  0.16964652, -0.11478166])

In [11]:
np.round(opt.get_weight_map(),3)

array([[ 1.117, -0.051, -0.038, -0.009,  0.006],
       [-0.051,  1.359,  0.338, -0.104, -0.34 ],
       [-0.038,  0.338,  1.507, -0.106,  0.041],
       [-0.009, -0.104, -0.106,  1.194, -0.075],
       [ 0.006, -0.34 ,  0.041, -0.075,  1.136]])

- To demostrate POET we simulate returns via a factor model of K factors such that K << p~n
- K: number of factors, p asset numbers, n time periods
-  about C: a good choice of C is chosen such that offdiagnal f-norm under is between 0.0 and its maximum

In [12]:
c = 0.3
n = 200
p = 100

rand = np.random.RandomState(1234)
temp = rand.normal(size = (p,n))
temp_u,temp_s,temp_vh =  LA.svd(temp)

true_K = 6
true_B = temp_u[:,:true_K]@np.diag(temp_s[:true_K])
true_F = temp_vh.T[:,:true_K]
errors = rand.normal(size = (p,n))*c
returns = true_B@true_F.T + errors

signal = pd.Series(np.ones(p))

In [13]:
opt = POET(C = 0.2)
opt.optimize(signal = signal,returns = returns)

backtest-2021-Feb-28 22:52:48-__init__: ShrinkToIdentity initialized
backtest-2021-Feb-28 22:52:48-optimize: Optimal K is 6
backtest-2021-Feb-28 22:52:48-optimize: offdiagnal f-norm under C=0.2 is 0.05308


array([0.01390297, 0.00604776, 0.007464  , 0.00991726, 0.01035616,
       0.0077557 , 0.01307507, 0.00813021, 0.00889522, 0.0100044 ,
       0.00909759, 0.0117812 , 0.00921658, 0.00972147, 0.00849731,
       0.00739705, 0.01212967, 0.00831121, 0.01300866, 0.00815562,
       0.01147961, 0.00915036, 0.01670194, 0.00795584, 0.0124574 ,
       0.01477995, 0.01013923, 0.01040963, 0.00567854, 0.01220117,
       0.01081098, 0.01016651, 0.00899532, 0.01096248, 0.00952144,
       0.00741788, 0.01364388, 0.00842481, 0.01177619, 0.01321186,
       0.00655681, 0.01252246, 0.01187519, 0.00956167, 0.0077062 ,
       0.00878962, 0.01045418, 0.01316003, 0.0083652 , 0.00747967,
       0.00536865, 0.01260353, 0.00965487, 0.00721568, 0.01363581,
       0.01386676, 0.01453695, 0.01153453, 0.00977448, 0.00709495,
       0.00851572, 0.00876866, 0.0083318 , 0.00653354, 0.00771605,
       0.01110796, 0.00927917, 0.00846805, 0.00669602, 0.01434645,
       0.01295281, 0.01070698, 0.01029886, 0.0103633 , 0.00539

In [14]:
np.diag(np.round(opt.get_weight_map(),3))

array([11.089, 10.607, 10.808, 10.512, 10.202,  9.835, 12.993, 10.796,
       10.166, 11.412, 11.497, 10.376, 13.089, 11.308, 11.676, 12.212,
       11.928, 11.524, 11.508, 10.114, 11.81 , 11.209, 13.82 , 11.702,
       10.99 , 11.008, 11.588, 12.516, 11.752, 12.104, 14.339, 11.168,
       11.074, 12.726, 11.017, 12.249, 12.429, 11.312, 12.512, 12.268,
       12.233, 13.356, 11.594, 11.129, 10.677, 11.546, 12.361, 12.115,
       11.319, 11.826, 10.02 , 11.568, 11.992, 10.125, 11.513, 10.621,
       11.561, 12.271, 11.672, 11.288, 10.921, 11.512, 10.146, 11.466,
       11.175, 11.741, 10.941, 10.175, 13.205, 11.349, 10.632, 11.668,
       11.24 , 10.942, 10.584, 10.561, 13.915,  9.933, 13.061, 13.255,
       11.38 , 10.995,  9.972, 11.236, 10.933, 10.453, 14.013, 14.514,
       11.368, 11.489, 10.861, 12.722, 10.602, 10.906, 12.202, 12.683,
       13.369, 10.237,  9.525, 11.66 ])